In [68]:
import cv2 
import glob 
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from skimage.transform import resize
import matplotlib.pyplot as plt
import pandas as pd
import re
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
import spacy
import string
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import precision_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.linear_model import Perceptron

In [8]:
def extract():
#Permet d'extraire les images et de les stocker dans une liste
#renvoie en deuxieme une liste contenant la taille de toutes les images
    taille_image=[]
    liste=[]
    liste_nom=[]
    for file in glob.glob("dataset/*"):
        img = cv2.imread(file)
        taille_image.append(img.shape[:2])
        liste.append(img)
        liste_nom.append(file)
    return liste_nom , liste, taille_image

In [9]:
liste_nom, liste_image, liste_taille = extract()

In [10]:
def transform_keypoints(liste_img):
#retourne une liste d'images transformées avec les keypoints ainsi qu'une liste des keypoints pour chaque image
    liste_img_transform=[]
    liste_kp=[]
    liste_des=[]
    for img in liste_img:
        gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        sift = cv2.SIFT_create(300)
        res = cv2.equalizeHist(gray)
        kp, des = sift.detectAndCompute(res, None)
        img=cv2.drawKeypoints(gray , 
                      kp , 
                      img , 
                      flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        liste_img_transform.append(img)
        liste_kp.append(kp)
        liste_des.append(des)
    return liste_img_transform, liste_kp, liste_des

In [11]:
liste_img_transform, liste_kp, liste_des = transform_keypoints(liste_image)

In [12]:
df = pd.DataFrame(zip(liste_nom,liste_kp, liste_des), columns=['Image','liste_kp','liste_des'])

In [13]:
df['label']=None

In [14]:
for i in range(df.shape[0]):
    df['label'][i]=' '.join((re.sub(r'[0-9]+', '', df['Image'][i]).split("dataset"))).replace('\\','').replace('.jpg','').replace('.jpeg','')
df.head()

,Image,liste_kp,liste_des,label
0,dataset\cloudy1.jpg,"(<KeyPoint 000001F2349181E0>, <KeyPoint 000001...","[[51.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 95.0, 99...",cloudy
1,dataset\cloudy10.jpg,"(<KeyPoint 000001F2010ED600>, <KeyPoint 000001...","[[55.0, 19.0, 20.0, 3.0, 0.0, 0.0, 0.0, 18.0, ...",cloudy
2,dataset\cloudy100.jpg,"(<KeyPoint 000001F23EC64B40>, <KeyPoint 000001...","[[71.0, 10.0, 2.0, 22.0, 15.0, 1.0, 0.0, 16.0,...",cloudy
3,dataset\cloudy11.jpg,"(<KeyPoint 000001F23EC68480>, <KeyPoint 000001...","[[65.0, 16.0, 0.0, 0.0, 2.0, 2.0, 1.0, 3.0, 16...",cloudy
4,dataset\cloudy12.jpg,"(<KeyPoint 000001F201144D80>, <KeyPoint 000001...","[[27.0, 78.0, 116.0, 22.0, 2.0, 2.0, 0.0, 0.0,...",cloudy


In [16]:
sift_keypoints_all    = np.concatenate(liste_des, axis=0)

In [17]:
sift_keypoints_all

array([[ 51.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   1.,   0.,  10.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   6.,  15., ...,   1.,   0.,   0.],
       [136.,  53.,   0., ...,   0.,   0.,   0.],
       [ 36.,   2.,   0., ...,   6.,   1.,   5.]], dtype=float32)

In [ ]:
liste_des

In [19]:
k=round(np.sqrt(len(sift_keypoints_all)))

In [20]:
k

322

In [21]:
kmeans = MiniBatchKMeans(n_clusters=k,
                         random_state=0,
                         batch_size=6,
                         max_iter=10).fit(sift_keypoints_all)
kmeans.cluster_centers_

C:\Users\Moog\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


array([[29.421846 , 12.220341 ,  4.747646 , ...,  7.954802 ,  2.4952908,
        14.962336 ],
       [16.872925 , 11.251381 ,  9.138126 , ...,  6.3066287,  8.353584 ,
        16.262428 ],
       [72.17088  , 26.418184 ,  5.305455 , ...,  2.8109086,  3.9818196,
        16.36     ],
       ...,
       [31.611105 , 19.64286  ,  8.920635 , ...,  8.730161 ,  6.34127  ,
        17.079367 ],
       [ 6.7568808,  9.325691 , 11.720184 , ...,  2.513762 ,  4.628441 ,
         9.844032 ],
       [24.056816 , 33.142056 , 30.511375 , ..., 12.204545 ,  9.346592 ,
        14.818182 ]], dtype=float32)

In [22]:
kmeans.predict(df['liste_des'][2])

array([106, 106, 258, 215, 150,  95, 142,  11, 114, 190, 246,  81, 207,
        45, 251, 230,  18, 185,  27, 172, 270,  13,  53, 149,  26, 316,
       101, 252,  16, 207, 169,  17,  44,   8, 108,  94, 276,  43, 189,
       189, 281, 138, 122,  53, 147, 213, 117, 116, 153, 103, 202,  65,
        20, 184, 221,  53, 184, 221,  20,   7,  75, 179,  94, 169, 207,
       290,  86,  56,  81, 144,  52, 283, 142, 245, 315, 183, 272, 125,
       151,  11,  79,   0, 289,   3, 232,  39,  45, 245,  71, 102,  33,
       256, 320, 251,  79, 179, 201, 140,  75, 287,  45, 138, 250, 168,
        20, 108, 216, 164, 138, 254, 273, 190,  86,  23,  37,  18, 238,
       210, 129, 237, 226, 246,  81,  22,  86, 238, 207, 136,  18, 215,
       198,  63, 234, 100, 143, 283, 215,  20, 222, 129, 192, 230, 293,
       141,  57, 254, 235, 309,   7, 233, 313, 293, 139, 242,  92, 158,
        45, 114,  20, 226,  43,  24, 188, 138,  14, 231, 118, 256, 235,
       309, 188, 282,  49, 317,  55,  94, 129, 219, 242, 203, 11

In [23]:
def build_histogram(kmeans, des, image_num): ## fonction qui nou permet de crée un histogramme par image 
    res = kmeans.predict(des)
    hist = np.zeros(len(kmeans.cluster_centers_))
    nb_des=len(des)
    if nb_des==0 : print("problème histogramme image  : ", image_num)
    for i in res:
        hist[i] += 1.0/nb_des
    return hist


In [24]:
hist_tous=[]
for i, des in enumerate(liste_des):
    
    hist_tous.append(build_histogram(kmeans, des, i))


In [25]:
X_features=pd.DataFrame(np.asarray(hist_tous))

In [26]:
X_features.shape

(400, 322)

In [27]:
X_features

,0,1,2,3,4,5,6,7,8,9,...,312,313,314,315,316,317,318,319,320,321
0,0.000000,0.013514,0.013514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000
1,0.000000,0.003322,0.013289,0.003322,0.000000,0.006645,0.006645,0.013289,0.009967,0.003322,...,0.000000,0.003322,0.003322,0.006645,0.003322,0.000000,0.003322,0.000000,0.000000,0.000000
2,0.010000,0.003333,0.000000,0.006667,0.000000,0.000000,0.000000,0.010000,0.006667,0.003333,...,0.000000,0.003333,0.003333,0.003333,0.006667,0.003333,0.000000,0.000000,0.006667,0.003333
3,0.000000,0.003333,0.000000,0.000000,0.003333,0.000000,0.003333,0.000000,0.016667,0.003333,...,0.000000,0.003333,0.003333,0.003333,0.006667,0.010000,0.000000,0.006667,0.000000,0.003333
4,0.000000,0.006667,0.000000,0.003333,0.003333,0.000000,0.003333,0.003333,0.000000,0.000000,...,0.000000,0.006667,0.006667,0.006667,0.010000,0.003333,0.000000,0.003333,0.013333,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.000000,0.000000,0.000000,0.003333,0.000000,0.000000,0.000000,0.000000,0.006667,0.003333,...,0.000000,0.003333,0.006667,0.000000,0.016667,0.000000,0.000000,0.003333,0.000000,0.003333
396,0.053333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013333,0.000000,...,0.000000,0.000000,0.000000,0.040000,0.006667,0.000000,0.000000,0.003333,0.000000,0.000000
397,0.009174,0.000000,0.000000,0.009174,0.000000,0.009174,0.009174,0.018349,0.000000,0.009174,...,0.018349,0.000000,0.000000,0.027523,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
398,0.016667,0.000000,0.006667,0.000000,0.003333,0.006667,0.000000,0.016667,0.010000,0.006667,...,0.000000,0.000000,0.000000,0.003333,0.003333,0.006667,0.000000,0.000000,0.000000,0.000000


In [28]:
df.head()

,Image,liste_kp,liste_des,label
0,dataset\cloudy1.jpg,"(<KeyPoint 000001F2349181E0>, <KeyPoint 000001...","[[51.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 95.0, 99...",cloudy
1,dataset\cloudy10.jpg,"(<KeyPoint 000001F2010ED600>, <KeyPoint 000001...","[[55.0, 19.0, 20.0, 3.0, 0.0, 0.0, 0.0, 18.0, ...",cloudy
2,dataset\cloudy100.jpg,"(<KeyPoint 000001F23EC64B40>, <KeyPoint 000001...","[[71.0, 10.0, 2.0, 22.0, 15.0, 1.0, 0.0, 16.0,...",cloudy
3,dataset\cloudy11.jpg,"(<KeyPoint 000001F23EC68480>, <KeyPoint 000001...","[[65.0, 16.0, 0.0, 0.0, 2.0, 2.0, 1.0, 3.0, 16...",cloudy
4,dataset\cloudy12.jpg,"(<KeyPoint 000001F201144D80>, <KeyPoint 000001...","[[27.0, 78.0, 116.0, 22.0, 2.0, 2.0, 0.0, 0.0,...",cloudy


In [29]:
le = preprocessing.LabelEncoder()
le.fit(df.label)
y=le.transform(df.label)

In [30]:
X_train, X_test, y_train, y_test = train_test_split (X_features, y , test_size=0.2, random_state=42)

## On utilise des algorithmes de classification 

In [56]:
resultat=[]
per=Perceptron()
per_model = per.fit(X_train, y_train)
resultat.append(("Perceptron",per_model.score(X_test, y_test)))
clf = MLPClassifier(random_state=1, max_iter=400).fit(X_train, y_train)
resultat.append(("MLP",clf.score(X_test, y_test)))
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)
resultat.append(("KNN",knn_model.score(X_test, y_test)))
svc = SVC()
svc_model = svc.fit(X_train, y_train)
resultat.append(("SVC",svc_model.score(X_test, y_test)))
nu_svc = NuSVC()
nu_svc_model = nu_svc.fit(X_train, y_train)
resultat.append(("NuSVC",nu_svc_model.score(X_test, y_test)))
linear_svc = LinearSVC()
linear_svc_model= linear_svc.fit(X_train, y_train)
resultat.append(("LinearSVC",linear_svc_model.score(X_test, y_test)))



C:\Users\Moog\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


In [57]:
resultat

[('Perceptron', 0.4),
 ('MLP', 0.6375),
 ('KNN', 0.525),
 ('SVC', 0.6375),
 ('NuSVC', 0.6375),
 ('LinearSVC', 0.55)]

In [55]:
#Test d'amélioration avec pipeline et standarscaler mais je n'obtiens pas de score plus élevée
clf_pipe=make_pipeline(StandardScaler(),nu_svc)
clf_pipe_model=clf.fit(X_train, y_train)
clf_pipe_model.score(X_test, y_test)


C:\Users\Moog\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


0.6375

Lancement de GridSearchCV sur nos 3 meilleurs modèles afin d'obtenir les meilleurs paramètres pour nos algorithmes et donc une amélioration de nos scores.

In [60]:
parameters = {'solver': ['lbfgs'], 'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(10, 15), 'random_state':[0,1,2,3,4,5,6,7,8,9]}
mlp_grid = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)

mlp_grid.fit(X_train, y_train)
print(mlp_grid.score(X_train, y_train))
print(mlp_grid.best_params_)

1.0
{'alpha': 0.1, 'hidden_layer_sizes': 13, 'max_iter': 1200, 'random_state': 1, 'solver': 'lbfgs'}


In [70]:

SVCpipe = Pipeline([('scale', StandardScaler()),
                   ('SVC',LinearSVC())])

param_grid = {'SVC__C':np.arange(0.01,100,10)}
linear_SVC = GridSearchCV(SVCpipe, param_grid,cv=5,return_train_score=True)
linear_SVC.fit(X_train,y_train)
print(linear_SVC.best_params_)
print(linear_svc.score(X_test, y_test))

C:\Users\Moog\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Moog\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Moog\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Moog\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Moog\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

{'SVC__C': 0.01}
0.55


In [66]:
parameters = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf', 'poly', 'sigmoid'],
              }
 
svc_grid = GridSearchCV(SVC(), parameters, n_jobs=-1)
svc_grid.fit(X_train, y_train)
print(svc_grid.score(X_test, y_test))
print(svc_grid.best_params_)

0.6
{'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}


In [64]:
parameters = {'n_neighbors':range(1,100),
               'weights':['uniform', 'distance'],
               'metric':['euclidean', 'manhattan'] }
knn_grid=GridSearchCV(KNeighborsClassifier(), parameters, cv=5, n_jobs=-1)
knn_grid.fit(X_train, y_train)
print(knn_grid.score(X_test, y_test))
print(knn_grid.best_params_)


0.6375
{'metric': 'manhattan', 'n_neighbors': 21, 'weights': 'distance'}


En conclusion le modèle MLP es le meilleur modèle de classification pour ce type de projet.